# Семинар 9

### Решающие деревья

<img src='tree_example.png'>

Сами по себе решающие деревья используются в машинном обучении относительно редко, однако очень распространены методы, основанные на их композиции - ансамблях (Random Forest, XGBoost, LightGBM).

##### Плюсы:

- интерпретируемость
- способность выделить самые важные признаки
- отсутствие потребности в серьезной предобработке данных

##### Минусы:

- склонность к переобучению
- неустойчивость - небольшие изменения в данных могут привести к сильному изменению в структуре дерева
- эвристичность обучения - как оптимизировать?

#####  Линейная модель vs "деревянная" модель (основанная на решающих деревьях):

- когда данные хорошо линейно разделимы, линейная модель лучше

- когда данные плохо линейно разделимы (много сложных нелинейных зависимостей в данных), "деревянная" модель лучше

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (15, 6)

### Переобучение

In [ ]:
def get_grid(data):
    x_min, x_max = data[:, 0].min() - 1, data[:, 0].max() + 1
    y_min, y_max = data[:, 1].min() - 1, data[:, 1].max() + 1
    return np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

In [ ]:
np.random.seed(13)
data_x = np.random.normal(size=(100, 2))
data_y = (data_x[:, 0] ** 2 + data_x[:, 1] ** 2) ** 0.5 # 
plt.figure(figsize=(8, 8))
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring')

In [ ]:
from sklearn.tree import DecisionTreeRegressor

clf = DecisionTreeRegressor(random_state=13)
clf.fit(data_x, data_y)

xx, yy = get_grid(data_x)

predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(8, 8))
plt.pcolormesh(xx, yy, predicted, cmap='spring')
plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=100, cmap='spring', edgecolor='k')
plt.show()

Посмотрим, как будет выглядеть разделение плоскости в зависимости от максимальной глубины дерева и минимального числа объектов в листе.

In [ ]:
plt.figure(figsize=(14, 14))
for i, max_depth in enumerate([2, 4, None]):
    for j, min_samples_leaf in enumerate([15, 5, 1]):
        clf = DecisionTreeRegressor(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=13)
        clf.fit(data_x, data_y)
        xx, yy = get_grid(data_x)
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
        plt.subplot2grid((3, 3), (i, j))
        plt.pcolormesh(xx, yy, predicted, cmap='spring')
        plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='spring', edgecolor='k')
        plt.title('max_depth=' + str(max_depth) + ' | min_samples_leaf=' + str(min_samples_leaf))

### Неустойчивость

Посмотрим, как будет меняться структура дерева, если брать для обучения разные 90%-ые подвыборки исходной выборки.

In [ ]:
plt.figure(figsize=(20, 6))
for i in range(3):
    clf = DecisionTreeRegressor(random_state=13)

    indecies = np.random.randint(data_x.shape[0], size=int(data_x.shape[0] * 0.9))
    clf.fit(data_x[indecies], data_y[indecies])
    xx, yy = get_grid(data_x)
    predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    plt.subplot2grid((1, 3), (0, i))
    plt.pcolormesh(xx, yy, predicted, cmap='winter')
    plt.scatter(data_x[:, 0], data_x[:, 1], c=data_y, s=30, cmap='winter', edgecolor='k')

### Практика

In [ ]:
boston = load_boston()

In [ ]:
print(boston['DESCR'])

In [ ]:
boston.keys()

In [ ]:
boston['feature_names']

In [ ]:
boston['data'].shape

In [ ]:
plt.hist(boston['target'])
plt.show()

In [ ]:
X = pd.DataFrame(boston['data'], columns=boston['feature_names'])
X['target'] = boston['target']

In [ ]:
X.head()

$R_m$ - множество объектов в разбиваемой вершине, $i$ - номер признака, по которому происходит разбиение, $t$ - порог разбиения.

Качество:

$$
Q(R_m, i, t) = H(R_m) - \frac{|R_\ell|}{|R_m|}H(R_\ell) - \frac{|R_r|}{|R_m|}H(R_r)
$$

$R_\ell$ - множество объектов в левом поддереве, $R_r$ - множество объектов в правом поддереве.

$H(R)$ - критерий информативности, с помощью которого можно оценить качество распределения целевой переменной среди объектов множества $R$.

_Вопрос. Что мы хотим сделать с $H(R)$ - минимизировать или максимизировать? А $Q(R_m, i, t)$?_

_Вопрос. Что можно использовать в качестве критерия информативности для регрессии?_

_Реализуйте подсчет качества разбиения._

In [ ]:
def H(R):
    pass


def split_node(R_m, feature, t):
    pass


def quality(R_m, feature, t):
    pass

_Переберите все возможные разбиения выборки по одному из признаков и постройте график качества разбиения в зависимости от значения порога._

In [ ]:
feature = # ʕ•ᴥ•ʔ
Q_array = []
for t in np.unique(X[feature]):
    Q_array.append(quality(X, feature, t))
plt.plot(Q_array)
plt.title(feature)
plt.xlabel('threshold')
plt.ylabel('quality')
plt.show()

_Напишите функцию, находящую оптимальное разбиение данной вершины по данному признаку._

In [ ]:
def get_optimal_split(R_m, feature):
    values = np.unique(R_m[feature])
    Q_array = np.array(list(map(lambda t: quality(R_m, feature, t), values)))
    opt_threshold = #ʕ•ᴥ•ʔ
    return opt_threshold, Q_array

In [ ]:
t, Q_array = get_optimal_split(X, feature)
plt.plot(Q_array)
plt.title(feature)
plt.xlabel('threshold')
plt.ylabel('quality')
plt.show()

_Постройте график качества разбиения (в зависимости от количества объектов в левом поддереве) для каждого из признаков. Найдите признак, показывающий наилучшее качество. Какой это признак? Каков порог разбиения и значение качества? Постройте график качества разбиения для данного признака в зависимости от значения порога._

In [ ]:
for f in boston['feature_names']:
    t, Q_array = get_optimal_split(X, f)
    print(f, t)
    plt.plot(Q_array, label=f + ' {0:.2f}'.format(Q_array.max()))
plt.legend(loc='best')
plt.show()

_Изобразите разбиение визуально. Для этого постройте диаграмму рассеяния целевой переменной в зависимости от значения найденного признака. Далее изобразите вертикальную линию, соответствующую разбиению. Почему это разбиение может быть лучшим? Как вы можете интерпретировать результат?_

In [ ]:
plt.scatter('#ʕ•ᴥ•ʔ')
plt.axvline('#ʕ•ᴥ•ʔ', color="red")
plt.xlabel('#ʕ•ᴥ•ʔ')
plt.ylabel('target')
plt.show()

### Многоклассовая классификация

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier 

In [ ]:
iris = load_iris()
X = pd.DataFrame(iris['data'], columns=iris['feature_names'])
y = iris['target']

In [ ]:
X.shape, y.shape

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

Понятно, что accuracy можно использовать как метрику качества не только в задаче бинарной классификации, но и в задаче многоклассовой классификации. Но что делать с precision, recall, f1? Можно сделать предсказание и воспользоваться какой-то из стратегий ниже.

#### Микро-усреднение (micro-average)

1) Считаются характеристики TP, FP, TN, FN для каждого класса.

2) Характеристики TP, FP, TN, FN усредняются по всем классам.

3) Метрики считаются по усредненным характеристикам.

Например, в случае precision:

$$
Precision = \frac{\sum\limits_{k}TP_k}{\sum\limits_{k}TP_k + \sum\limits_{k}FP_k},
$$

где $TP_k$ и $FP_k$ - TP и FP для класса $k$ соответственно.

#### Макро-усреднение (macro-average)

1) Считаются характеристики TP, FP, TN, FN для каждого класса.

2) Считаются метрики для каждого класса.

3) Итоговые метрики равны средним посчитанных метрик.

В данном случае precision считается так:

$$
Precision = \frac{\sum\limits_{k}Precision_k}{K},
$$

где $Precision_k$ - значение precision для класса $k$, а $K$ - общее число классов.

In [ ]:
def get_scores(estimator, X=X_test, y=y_test):
    y_pred = estimator.predict(X)

    accuracy = accuracy_score(y_test, y_pred)
    precision_micro = precision_score(y_test, y_pred, average='micro')
    precision_macro = precision_score(y_test, y_pred, average='macro')
    recall_micro = recall_score(y_test, y_pred, average='micro')
    recall_macro = recall_score(y_test, y_pred, average='macro')
    f1_micro = f1_score(y_test, y_pred, average='micro')
    f1_macro = f1_score(y_test, y_pred, average='macro')
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    columns = ['accuracy', 'precision_micro', 'precision_macro', 'recall_micro', 'recall_macro', 'f1_micro', 'f1_macro']
    results = pd.DataFrame(
        [accuracy, precision_micro, precision_macro, recall_micro, recall_macro, f1_micro, f1_macro],
        index=columns
    ).T
    conf_matrix = pd.DataFrame(
        conf_matrix,
        columns=['Predicted: 0', 'Predicted: 1', 'Predicted: 2'],
        index=['Actual: 0', 'Actual: 1', 'Actual: 2']
    )
    
    return results, conf_matrix

In [ ]:
dt = DecisionTreeClassifier(max_depth=2)
dt.fit(X_train, y_train)
dt_res, dt_conf = get_scores(dt)
dt_res

In [ ]:
dt_conf

In [ ]:
lr = LogisticRegression(C=1)
lr.fit(X_train, y_train)
lr_res, lr_conf = get_scores(lr)
lr_res

In [ ]:
lr_conf

Некоторые классификаторы (например, логистическая регрессия, решающее дерево) могут быть использованы не только для задач бинарной классификации, но и для задач многоклассовой классификации. В общем же случае для решения задачи многоклассовой классификации можно свести ее к задачам бинарной разными методами.

#### One-vs-All

Разбиваем задачу многоклассовой классификации с $K$ классами на $K$ задач бинарной классификации, где обучаем классификаторы, которые предсказывают ответы вида "принадлежит классу $k$ / не принадлежит классу $k$".

k-я задача:

- $X = \left(x_i, [y_i = k] - [y_i \neq k]\right)_{i=1}^\ell$

- классификатор $a_k(x) = \operatorname{sign}\langle w_k, x\rangle$

В качестве предсказания выбираем класс, соответствующий наиболее "уверенному" в своем классе классификатору:

$$
a(x) = \arg\max_{k\in\{1, \ldots, K\}}\langle w_k, x\rangle
$$

#### One-vs-One

Разбиваем задачу многоклассовой классификации с $K$ классами на $\frac{K(K - 1)}{2}$ задач бинарной классификации, где для каждой пары классов $(k, k')$ обучаем классификатор, который предсказывает ответы вида "принадлежит классу $k$ / принадлежит классу $k'$".

Задача для $(k, k')$:

- $X = \left(x_i, [y_i = k] - [y_i \neq k']\right)_{i=1}^\ell$

- классификатор $a_{k, k'}(x) = \operatorname{sign}\langle w_{k, k'}, x\rangle$

В качестве предсказания выбираем класс, который чаще всего выбирался бинарными классификаторами:

$$
a(x) = \arg\max_{k\in\{1, \ldots, K\}}\left|\{k':a_{k, k'}(x) = 1\}\right|
$$

In [ ]:
ovr = OneVsRestClassifier(lr)
ovr.fit(X_train, y_train)
ovr_res, ovr_conf = get_scores(ovr)
ovr_res

In [ ]:
ovr_conf

In [ ]:
ovr.estimators_

In [ ]:
ovo = OneVsOneClassifier(lr)
ovo.fit(X_train, y_train)
ovo_res, ovo_conf = get_scores(ovo)
ovo_res

In [ ]:
ovo_conf

In [ ]:
ovo.estimators_

### Задания для самостоятельного решения

1) https://github.com/esokolov/ml-minor-hse/blob/master/colloquium-2017/colloquium_minor_problems_trees.ipynb

2) Сколько задач бинарной классификации придется решить, чтобы решить задачу многоклассовой классификации с 5 классами с помощью методов One-vs-All и One-vs-One?

3) Влияет ли нормирование признаков на качество решающего дерева?

4) Приведите примеры критериев информативности для регрессии и классификации.

5) Почему для любой выборки можно построить решающее дерево, имеющее нулевую ошибку на ней?

6) Изучите гиперпараметры `DecisionTreeRegressor` и `DecisionTreeClassifier` в документации `sklearn`. Какие из них, на ваш взгляд, ключевые, и требуют особого внимания при подборе гиперпараметров с помощью кросс-валидации?